In [ ]:
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# LOAD TRAINED MODEL
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)

model.load_state_dict(torch.load("resnet18_85_8.pth", map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

print(" Model Loaded Successfully")

 Model Loaded Successfully


In [ ]:
mean = torch.tensor([0.5, 0.5, 0.5]).view(1,3,1,1).to(device)
std = torch.tensor([0.5, 0.5, 0.5]).view(1,3,1,1).to(device)

In [ ]:
def evaluate_cifar10c(corruption, severity, batch_size=256):

    data = np.load(f"{corruption}.npy")
    labels = np.load("labels.npy")

    start = (severity-1) * 10000
    end = severity * 10000

    images = data[start:end]
    labels_subset = labels[start:end]

    images = torch.tensor(images).permute(0,3,1,2).float() / 255.0
    labels_subset = torch.tensor(labels_subset)

    loader = DataLoader(
        TensorDataset(images, labels_subset),
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in loader:

            x = x.to(device)
            y = y.to(device)

            # normalize
            x = (x - mean) / std

            outputs = model(x)
            preds = outputs.argmax(1)

            correct += preds.eq(y).sum().item()
            total += y.size(0)

    acc = 100 * correct / total
    return acc

In [ ]:
# RUN STRUCTURED EXPERIMENT
corruptions = [
    "gaussian_noise",
    "motion_blur",
    "fog",
    "brightness",
    "contrast",
    "jpeg_compression"
]

severities = [3, 5]

clean_acc = 85.84
results = []

print("\n Running CIFAR-10-C Robustness Evaluation...\n")


 Running CIFAR-10-C Robustness Evaluation...



In [ ]:
for corr in corruptions:
    for sev in severities:

        acc = evaluate_cifar10c(corr, sev)
        drop = clean_acc - acc

        results.append((corr, sev, acc, drop))

        print(f"{corr.upper()} | Severity {sev}")
        print(f"Accuracy: {acc:.2f}%")
        print(f"Performance Drop: {drop:.2f}%\n")

GAUSSIAN_NOISE | Severity 3
Accuracy: 61.56%
Performance Drop: 24.28%

GAUSSIAN_NOISE | Severity 5
Accuracy: 56.03%
Performance Drop: 29.81%

MOTION_BLUR | Severity 3
Accuracy: 55.60%
Performance Drop: 30.24%

MOTION_BLUR | Severity 5
Accuracy: 51.68%
Performance Drop: 34.16%

FOG | Severity 3
Accuracy: 64.27%
Performance Drop: 21.57%

FOG | Severity 5
Accuracy: 47.69%
Performance Drop: 38.15%

BRIGHTNESS | Severity 3
Accuracy: 69.65%
Performance Drop: 16.19%

BRIGHTNESS | Severity 5
Accuracy: 64.26%
Performance Drop: 21.58%

CONTRAST | Severity 3
Accuracy: 62.37%
Performance Drop: 23.47%

CONTRAST | Severity 5
Accuracy: 42.10%
Performance Drop: 43.74%

JPEG_COMPRESSION | Severity 3
Accuracy: 68.86%
Performance Drop: 16.98%

JPEG_COMPRESSION | Severity 5
Accuracy: 66.94%
Performance Drop: 18.90%



In [ ]:
df = pd.DataFrame(
    results,
    columns=["Corruption","Severity","Accuracy (%)","Drop (%)"]
)

df = df.sort_values(by="Drop (%)", ascending=False)

print("\n==============================")
print(" FINAL ROBUSTNESS TABLE")
print("==============================\n")

display(df)

print("\n Mean Performance Drop:",
      round(df["Drop (%)"].mean(),2), "%")


 FINAL ROBUSTNESS TABLE



,Corruption,Severity,Accuracy (%),Drop (%)
9,contrast,5,42.10,43.74
5,fog,5,47.69,38.15
3,motion_blur,5,51.68,34.16
2,motion_blur,3,55.60,30.24
1,gaussian_noise,5,56.03,29.81
0,gaussian_noise,3,61.56,24.28
8,contrast,3,62.37,23.47
7,brightness,5,64.26,21.58
4,fog,3,64.27,21.57
11,jpeg_compression,5,66.94,18.90



 Mean Performance Drop: 26.59 %


In [ ]:
torch.save(model.state_dict(), "resnet18_cifar10_85_baseline.pth")